<a href="https://colab.research.google.com/github/ZaneZaiontz/YosAI/blob/main/YosAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports/Opening and reading in data

In [ ]:
%tensorflow_version 2.x

from keras.preprocessing import sequence
import tensorflow as tf
import random as rand
import numpy as np
import keras, os

def readFiles():
    file = open('./zhaoClean.txt', 'r')
    lines = file.readlines()
    file.close()
    file = open('./zhaoClean.txt', 'r')
    return file.read().lower(), lines

text, lines = readFiles()

Character Encoding/Sequence mapping (setting up future predictive modelling)


In [ ]:
vocab = sorted(set(text))
# Unique ID 
charToInt = {j:i for i, j in enumerate(vocab)}
intToChar = np.array(vocab)

tmpData = np.array([charToInt[i] for i in text])
charData = tf.data.Dataset.from_tensor_slices(tmpData)
# training char length
sequenceLen = 64
sequences = charData.batch(sequenceLen+1, drop_remainder=True)

def splitInput(chunk):  
    inText = chunk[:-1]  
    toText = chunk[1:] 
    return inText, toText 

dataset = sequences.map(splitInput)  

Build Model/Shuffle Data

In [ ]:
# Training data
sizeOfEmbed = 256
sizeOfRNN = 1024
sizeOfBuff = 10000
sizeofBatch = 64
sizeOfVocab = len(vocab)

data = dataset.shuffle(sizeOfBuff).batch(sizeofBatch, drop_remainder=True)
def buildModel(vocabSize, embedSize, rnnSize, batchSize):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabSize, embedSize, batch_input_shape=[batchSize, None]),
    tf.keras.layers.LSTM(rnnSize, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocabSize)])
  return model
model = buildModel(sizeOfVocab, sizeOfEmbed, sizeOfRNN, sizeofBatch)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           15360     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 60)            61500     
Total params: 5,323,836
Trainable params: 5,323,836
Non-trainable params: 0
_________________________________________________________________


Compile/Loss Function


Checkpoints/Fitting

In [ ]:
numOfEpochs = 128

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

checkLocation = './trainingData'
checkPre = os.path.join(checkLocation, "CKPT_{epoch}")

checkBack=tf.keras.callbacks.ModelCheckpoint(filepath=checkPre, save_weights_only=True)
history = model.fit(data, epochs=numOfEpochs, callbacks=[checkBack])

model = buildModel(sizeOfVocab, sizeOfEmbed, sizeOfRNN, batchSize=1)
model.load_weights(tf.train.latest_checkpoint(checkLocation))
model.build(tf.TensorShape([1, None]))

Epoch 1/128
61/61 [==============================] - 6s 47ms/step - loss: 3.2538
Epoch 2/128
61/61 [==============================] - 3s 47ms/step - loss: 2.5852
Epoch 3/128
61/61 [==============================] - 3s 47ms/step - loss: 2.3512
Epoch 4/128
61/61 [==============================] - 3s 47ms/step - loss: 2.2033
Epoch 5/128
61/61 [==============================] - 3s 47ms/step - loss: 2.0959
Epoch 6/128
61/61 [==============================] - 3s 47ms/step - loss: 1.9915
Epoch 7/128
61/61 [==============================] - 3s 46ms/step - loss: 1.9113
Epoch 8/128
61/61 [==============================] - 3s 46ms/step - loss: 1.8439
Epoch 9/128
61/61 [==============================] - 3s 47ms/step - loss: 1.7726
Epoch 10/128
61/61 [==============================] - 3s 46ms/step - loss: 1.7175
Epoch 11/128
61/61 [==============================] - 3s 46ms/step - loss: 1.6646
Epoch 12/128
61/61 [==============================] - 3s 46ms/step - loss: 1.6195
Epoch 13/128
61/61 [=====

Output

In [ ]:
# createPoem generates 3 lines of words
def createPoem(model):
  newL = 0

  startInt = (rand.randint(0, len(lines)))/3
  startLine = lines[int(startInt)]
  startLine = startLine.lower()

  checkInput = [charToInt[s] for s in startLine]
  checkInput = tf.expand_dims(checkInput, 0)
  generatedWord = []
  model.reset_states()

  # For haiku format
  while newL < 3:
      # Predict based off previous sequence
      predictions = model(checkInput)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions
      idPredict = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      checkInput = tf.expand_dims([idPredict], 0)
      if (intToChar[idPredict] == '\n'):
        newL += 1
      if (newL >= 3):
        generatedWord.append(intToChar[idPredict])
        break
      generatedWord.append(intToChar[idPredict])
  return (''.join(generatedWord))

print(createPoem(model))



tree
sombine colors
in the morning ed empty

